In [25]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.stats import pearsonr
import sys
from pathlib import Path

In [27]:
import pandas as pd
import numpy as np

folder = r'C:\Users\u1355\OneDrive\Desktop\Data Viz\Thesis Website\birds\Eared Grebe Mar- May\\'
files = glob.glob(folder + '*Bear*.txt')
files.sort()
print("Files found:", files)


df_all = pd.DataFrame(columns=['year', 'march_avg', 'april_avg', 'may_avg'])

for file in files:
    print(file)
    df = pd.read_table(file)
    i1 = df.index[df['Unnamed: 0'].eq('Average Count')]
    df = df[i1[0] + 1:]
    df = df[df['Week starting on...'].str.contains('checklist', na=False) == False]
    df.replace(0, np.nan, inplace=True)

    # Extract years from 'Week starting on...' column
    years = df['Week starting on...'].str.extract('(\d+)')[0].astype(float).dropna().unique().astype(int)    

    for year in years:
        # Filter data for the current year
        df_year = df[df['Week starting on...'].str.contains(str(year), na=False)] # Apply the fix here
        
        # Reshape data and calculate monthly averages
        df_melted = df_year.melt(id_vars=['Week starting on...'], var_name='date', value_name='count')
        df_melted['month'] = pd.to_numeric(df_melted['date'].str.extract('(\d+)/\d+')[0], errors='coerce').astype('Int64')
        monthly_avgs = df_melted.groupby('month')['count'].mean()

        # Store monthly averages in df_all
        df_all = pd.concat([df_all, pd.DataFrame({'year': [year],
                                                  'march_avg': [monthly_avgs.get(3, np.nan)],
                                                  'april_avg': [monthly_avgs.get(4, np.nan)],
                                                  'may_avg': [monthly_avgs.get(5, np.nan)]})])

# Convert df_all to xarray
df_all_xr = df_all.set_index('year').to_xarray()
df_all_xr = df_all_xr.rename_dims(year='time')
df_all_xr = df_all_xr.rename({'year': 'time'})

print(df_all_xr)

Files found: ['C:\\Users\\u1355\\OneDrive\\Desktop\\Data Viz\\Thesis Website\\birds\\Eared Grebe Mar- May\\Eared Grebe Bear River 2004-2008.txt', 'C:\\Users\\u1355\\OneDrive\\Desktop\\Data Viz\\Thesis Website\\birds\\Eared Grebe Mar- May\\Eared Grebe Bear River 2009-2013.txt', 'C:\\Users\\u1355\\OneDrive\\Desktop\\Data Viz\\Thesis Website\\birds\\Eared Grebe Mar- May\\Eared Grebe Bear River 2014-2018.txt', 'C:\\Users\\u1355\\OneDrive\\Desktop\\Data Viz\\Thesis Website\\birds\\Eared Grebe Mar- May\\Eared Grebe Bear River 2019-2023.txt']
C:\Users\u1355\OneDrive\Desktop\Data Viz\Thesis Website\birds\Eared Grebe Mar- May\Eared Grebe Bear River 2004-2008.txt


C:\Users\u1355\AppData\Local\Temp\ipykernel_34012\3308746635.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat([df_all, pd.DataFrame({'year': [year],


C:\Users\u1355\OneDrive\Desktop\Data Viz\Thesis Website\birds\Eared Grebe Mar- May\Eared Grebe Bear River 2009-2013.txt
C:\Users\u1355\OneDrive\Desktop\Data Viz\Thesis Website\birds\Eared Grebe Mar- May\Eared Grebe Bear River 2014-2018.txt
C:\Users\u1355\OneDrive\Desktop\Data Viz\Thesis Website\birds\Eared Grebe Mar- May\Eared Grebe Bear River 2019-2023.txt
<xarray.Dataset> Size: 640B
Dimensions:    (time: 20)
Coordinates:
  * time       (time) object 160B 2004 2005 2006 2007 ... 2020 2021 2022 2023
Data variables:
    march_avg  (time) float64 160B nan 161.0 nan 1.0 nan ... 4.778 2.8 2.333 nan
    april_avg  (time) float64 160B nan 1.872e+03 304.0 13.67 ... 25.87 16.81 nan
    may_avg    (time) float64 160B nan nan 738.0 21.88 ... 3.25 18.13 28.27 nan


C:\Users\u1355\AppData\Local\Temp\ipykernel_34012\3308746635.py:41: UserWarning: rename 'year' to 'time' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  df_all_xr = df_all_xr.rename({'year': 'time'})


In [29]:
df_all_df = df_all_xr.to_dataframe()

In [30]:
df_all_df

,march_avg,april_avg,may_avg
time,,,
2004,NaN,NaN,NaN
2005,161.000000,1871.500000,NaN
2006,NaN,304.000000,738.000000
2007,1.000000,13.666667,21.875000
2008,NaN,56.500000,3.000000
2009,1.000000,5.500000,3.750000
2010,NaN,59.250000,11.666667
2011,4.000000,83.208333,22.950000
2012,4.000000,1138.000000,108.200000


In [34]:
df_all_df.to_json(r'C:\Users\u1355\OneDrive\Desktop\Data Viz\Thesis Website\birds\eg_geojsons\EG_Spring_Avgs.json')